## Training CBoW-Modell

Dieses Notebook ist Teil des [AI for Beginners Curriculum](http://aka.ms/ai-beginners)

In diesem Beispiel werden wir ein CBoW-Sprachmodell trainieren, um unseren eigenen Word2Vec-Einbettungsraum zu erstellen. Als Textquelle verwenden wir den AG News-Datensatz.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Zuerst laden wir unser Dataset und definieren Tokenizer und Vokabular. Wir setzen `vocab_size` auf 5000, um die Berechnungen etwas zu begrenzen.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW-Modell

CBoW lernt, ein Wort basierend auf den $2N$ benachbarten Wörtern vorherzusagen. Zum Beispiel, wenn $N=1$, erhalten wir die folgenden Paare aus dem Satz *I like to train networks*: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks). Hier ist das erste Wort das benachbarte Wort, das als Eingabe verwendet wird, und das zweite Wort ist das, das wir vorhersagen.

Um ein Netzwerk zu erstellen, das das nächste Wort vorhersagt, müssen wir das benachbarte Wort als Eingabe bereitstellen und die Wortnummer als Ausgabe erhalten. Die Architektur des CBoW-Netzwerks sieht wie folgt aus:

* Das Eingabewort wird durch die Embedding-Schicht geleitet. Diese Embedding-Schicht wird unser Word2Vec-Embedding sein, daher definieren wir sie separat als die Variable `embedder`. In diesem Beispiel verwenden wir eine Embedding-Größe von 30, obwohl Sie möglicherweise mit höheren Dimensionen experimentieren möchten (echtes Word2Vec hat 300).
* Der Embedding-Vektor wird dann an eine lineare Schicht weitergegeben, die das Ausgabewort vorhersagt. Daher hat sie `vocab_size` Neuronen.

Für die Ausgabe: Wenn wir `CrossEntropyLoss` als Verlustfunktion verwenden, müssen wir auch nur die Wortnummern als erwartete Ergebnisse bereitstellen, ohne One-Hot-Encoding.


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## Vorbereitung der Trainingsdaten

Nun programmieren wir die Hauptfunktion, die CBoW-Wortpaare aus Text berechnet. Diese Funktion ermöglicht es uns, die Fenstergröße festzulegen und gibt ein Set von Paaren zurück – Eingabe- und Ausgabewort. Beachten Sie, dass diese Funktion sowohl auf Wörter als auch auf Vektoren/Tensoren angewendet werden kann – was es uns ermöglicht, den Text zu kodieren, bevor er an die Funktion `to_cbow` übergeben wird.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


Lass uns den Trainingsdatensatz vorbereiten. Wir werden alle Nachrichten durchgehen, `to_cbow` aufrufen, um die Liste der Wortpaare zu erhalten, und diese Paare zu `X` und `Y` hinzufügen. Aus Zeitgründen werden wir nur die ersten 10.000 Nachrichten berücksichtigen - du kannst die Einschränkung leicht entfernen, falls du mehr Zeit hast und bessere Einbettungen erhalten möchtest :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

Wir werden diese Daten auch in einen Datensatz umwandeln und einen Datenlader erstellen:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

Wir werden diese Daten auch in einen Datensatz umwandeln und einen Datenlader erstellen:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

Jetzt beginnen wir mit dem eigentlichen Training. Wir verwenden den `SGD`-Optimierer mit einer ziemlich hohen Lernrate. Du kannst auch andere Optimierer ausprobieren, wie zum Beispiel `Adam`. Wir werden zunächst für 10 Epochen trainieren – und du kannst diese Zelle erneut ausführen, wenn du einen noch geringeren Verlust erzielen möchtest.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Ausprobieren von Word2Vec

Um Word2Vec zu verwenden, extrahieren wir die Vektoren, die den Wörtern in unserem Vokabular entsprechen:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

Lass uns sehen, wie das Wort **Paris** beispielsweise in einen Vektor codiert wird:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


Es ist interessant, Word2Vec zu verwenden, um nach Synonymen zu suchen. Die folgende Funktion gibt die `n` nächsten Wörter zu einer gegebenen Eingabe zurück. Um sie zu finden, berechnen wir die Norm von $|w_i - v|$, wobei $v$ der Vektor ist, der unserem Eingabewort entspricht, und $w_i$ die Kodierung des $i$-ten Wortes im Vokabular ist. Anschließend sortieren wir das Array und geben die entsprechenden Indizes mit `argsort` zurück, wobei wir die ersten `n` Elemente der Liste nehmen, die die Positionen der nächsten Wörter im Vokabular kodieren.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## Erkenntnis

Mit cleveren Techniken wie CBoW können wir ein Word2Vec-Modell trainieren. Du kannst auch versuchen, ein Skip-Gram-Modell zu trainieren, das darauf ausgelegt ist, das benachbarte Wort basierend auf dem zentralen Wort vorherzusagen, und sehen, wie gut es funktioniert.



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
